In [1]:
# do the imports

import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer

from sklearn.decomposition import NMF

import matplotlib.pyplot as plt
import seaborn as sns

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from nltk.corpus import stopwords, twitter_samples
nltk_stops = stopwords.words('english')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

import copy


import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df_ca    = pd.read_csv('../project_5_second/cleaned/paid_ca.csv')
df_chile = pd.read_csv('../project_5_second/cleaned/paid_chile.csv')
df_nepal = pd.read_csv('../project_5_second/cleaned/paid_nepal.csv')
df_pak   = pd.read_csv('../project_5_second/cleaned/paid_pak.csv')

df_supra = pd.concat([df_ca, df_chile, df_nepal, df_pak], axis= 0)

In [3]:
df_supra.head(1)

tweet_id                                         tweet_text                                label
0  '504075795435094016'  bridges overpasses earn approval after norther...  infrastructure_and_utilities_damage

In [4]:
def binarize_label(df):
    df['label_is_relevent'] = df['label'].apply(lambda x: 0 if x == 'not_related_or_irrelevant' else 1)
    return df

In [5]:
def cleaning(df, column):
    
#     tokenizing the strings and cleaning punctuation
    tokenizer = RegexpTokenizer('\s+', gaps=True)
    for i in range(df.shape[0]):
        text = df[column][i]
        tokens = tokenizer.tokenize(text)
        clean_tokens = []
        for token in tokens:
            token = token.replace(",", 
                          "").replace("?", 
                            "").replace("!", 
                              "").replace("'", 
                                "").replace(".",
                                  "").replace("\'t", 
                                   "").replace("#",
                                    "").replace("RT",
                                        "").lower()
            token = token.encode('ascii', 'ignore').decode('ascii')
            clean_tokens.append(token)

        seperator = ' '
        df[column][i] = seperator.join(clean_tokens)
    return df


In [6]:
df_ca    = binarize_label(df_ca)
df_chile = binarize_label(df_chile)
df_nepal = binarize_label(df_nepal)
df_pak   = binarize_label(df_pak)
df_supra = binarize_label(df_supra)

In [7]:
df_supra.head(1)

tweet_id                                         tweet_text                                label  label_is_relevent
0  '504075795435094016'  bridges overpasses earn approval after norther...  infrastructure_and_utilities_damage                  1

In [8]:
df_supra['label_is_relevent'].unique()

array([1, 0])

In [9]:
df_supra['label_is_relevent'].value_counts(normalize= True)

1    0.890102
0    0.109898
Name: label_is_relevent, dtype: float64

In [10]:
X = df_supra['tweet_text']
y = df_supra['label_is_relevent']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify= y, random_state= 42)

In [12]:
pipe_tvec_nb = Pipeline([
    ('tvec', TfidfVectorizer()),
    ('multi_nb', MultinomialNB())
])

params_tvec_nb = {
    'tvec__max_features': [1000, 2500, 5000, 10_000, 25_000, 40_000, 60_000],
    'tvec__stop_words': [None, nltk_stops],
    'tvec__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)],
    'multi_nb__alpha': [1.0, 0.0, 2.0]
}

gs_tvec_nb = GridSearchCV(estimator= pipe_tvec_nb, 
                          param_grid= params_tvec_nb,
                          cv= 3)

In [13]:
gs_tvec_nb.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('tvec',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

In [14]:
gs_tvec_nb.best_params_

{'multi_nb__alpha': 0.0,
 'tvec__max_features': 25000,
 'tvec__ngram_range': (1, 3),
 'tvec__stop_words': ['i',
  'me',
  'my',
  'myself',
  'we',
  'our',
  'ours',
  'ourselves',
  'you',
  "you're",
  "you've",
  "you'll",
  "you'd",
  'your',
  'yours',
  'yourself',
  'yourselves',
  'he',
  'him',
  'his',
  'himself',
  'she',
  "she's",
  'her',
  'hers',
  'herself',
  'it',
  "it's",
  'its',
  'itself',
  'they',
  'them',
  'their',
  'theirs',
  'themselves',
  'what',
  'which',
  'who',
  'whom',
  'this',
  'that',
  "that'll",
  'these',
  'those',
  'am',
  'is',
  'are',
  'was',
  'were',
  'be',
  'been',
  'being',
  'have',
  'has',
  'had',
  'having',
  'do',
  'does',
  'did',
  'doing',
  'a',
  'an',
  'the',
  'and',
  'but',
  'if',
  'or',
  'because',
  'as',
  'until',
  'while',
  'of',
  'at',
  'by',
  'for',
  'with',
  'about',
  'against',
  'between',
  'into',
  'through',
  'during',
  'before',
  'after',
  'above',
  'below',
  'to',
  'from

In [15]:
gs_tvec_nb.score(X_train, y_train)

0.991232190386723

In [16]:
gs_tvec_nb.score(X_test, y_test)

0.9009389671361502

In [17]:
# cross_val_score(gs_tvec_nb, X_train, y_train, cv= 3).mean()

In [18]:
df_supra_extended = df_supra.apply(copy.deepcopy)

In [19]:
df_supra_extended.head(1
                      )

tweet_id                                         tweet_text                                label  label_is_relevent
0  '504075795435094016'  bridges overpasses earn approval after norther...  infrastructure_and_utilities_damage                  1

In [20]:
df_supra_extended.shape

(8517, 4)

In [21]:
df_supra_extended['label_is_relevent'].value_counts(normalize= True)

1    0.890102
0    0.109898
Name: label_is_relevent, dtype: float64

In [22]:
df_supra_extended['label_is_relevent'].value_counts(normalize= False)

1    7581
0     936
Name: label_is_relevent, dtype: int64

In [23]:
tw_txt_list = []
strings = twitter_samples.strings('tweets.20150430-223406.json')
for string in strings[:20_000]:
    tw_txt_list.append(string)
    
tw_txt_list

['RT @KirkKus: Indirect cost of the UK being in the EU is estimated to be costing Britain £170 billion per year! #BetterOffOut #UKIP',
 'VIDEO: Sturgeon on post-election deals http://t.co/BTJwrpbmOY',
 'RT @LabourEoin: The economy was growing 3 times faster on the day David Cameron became Prime Minister than it is today.. #BBCqt http://t.co…',
 'RT @GregLauder: the UKIP east lothian candidate looks about 16 and still has an msn addy http://t.co/7eIU0c5Fm1',
 "RT @thesundaypeople: UKIP's housing spokesman rakes in £800k in housing benefit from migrants.  http://t.co/GVwb9Rcb4w http://t.co/c1AZxcLh…",
 'RT @Nigel_Farage: Make sure you tune in to #AskNigelFarage tonight on BBC 1 at 22:50! #UKIP http://t.co/ogHSc2Rsr2',
 'RT @joannetallis: Ed Milliband is an embarrassment. Would you want him representing the UK?!  #bbcqt vote @Conservatives',
 "RT @abstex: The FT is backing the Tories. On an unrelated note, here's a photo of FT leader writer Jonathan Ford (next to Boris) http://t.c…",
 "RT

In [24]:
len(tw_txt_list)

20000

In [25]:
tw_tw_id = ['007'] * 20_000
tw_label = ['dont care']* 20_000
tw_lbl_rel = [0]* 20_000

In [26]:
riff_raff = {'tweet_id':tw_tw_id, 'tweet_text':tw_txt_list, 'label': tw_label, 'label_is_relevent':tw_lbl_rel}
df_riff_raff = pd.DataFrame(riff_raff)

In [27]:
df_riff_raff.head(1)

tweet_id                                         tweet_text      label  label_is_relevent
0      007  RT @KirkKus: Indirect cost of the UK being in ...  dont care                  0

In [28]:
df_riff_raff = cleaning(df_riff_raff, 'tweet_text')

In [29]:
df_supra_extended_new = pd.concat([df_supra_extended, df_riff_raff], axis= 0)

In [30]:
df_supra_extended_new.shape

(28517, 4)

In [31]:
lr = LogisticRegression()

In [32]:
X = df_supra_extended_new['tweet_text']
y = df_supra_extended_new['label_is_relevent']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify= y, random_state= 42)

In [34]:
tvec = TfidfVectorizer(max_features= 25000,
                       stop_words= None,
                       ngram_range= (1, 2))

tvec.fit(X_train, y_train)
X_train_tvec = tvec.transform(X_train).todense()
X_test_tvec = tvec.transform(X_test).todense()

In [35]:
lr.fit(X_train_tvec, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
lr.score(X_train_tvec, y_train), lr.score(X_test_tvec, y_test)

(0.9792864824426053, 0.9743338008415148)

In [38]:
df_trial = pd.read_csv('../project_5_second/cleaned/earthquake.csv')

In [41]:
df_trial.head(1)

Unnamed: 0         created                                               text                                           location hashtags  location#  Unnamed: 6
0           0  10/28/19 17:11  Ml 2.54 earthquake (reviewed) occured at 2019-...  {'type': 'Point', 'coordinates': [35.13, -95.3...       []          0           1

In [42]:
df_trial = cleaning(df_trial, 'text')

In [43]:
X = df_trial['text']
y = df_trial['Unnamed: 6']

In [44]:


X_tvec = tvec.transform(X).todense()


In [45]:
lr.score(X_tvec, y)

0.9236641221374046

In [46]:
y.mean()

0.8412213740458016

In [47]:
X.shape

(655,)